<a href="https://colab.research.google.com/github/s1230038/autoencoder/blob/master/AutoencoderInTensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls -l
#!rm -r *
#!ls -l

total 4
drwxr-xr-x 1 root root 4096 Dec 10 17:34 sample_data


In [0]:
from google.colab import files
# !zip -r autoencoder.zip images latent_default.csv logs
# files.download('autoencoder.zip')

In [3]:
# Setup ngrok and run TensorBoard on Colab
# https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2018-12-14 07:45:01--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.165.51.142, 52.44.92.122, 54.174.228.92, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.165.51.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  21.8MB/s    in 0.2s    

2018-12-14 07:45:02 (21.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://bc7f2033.ngrok.io


In [4]:
# Simple MNIST Autoencoder in TensorFlow
# https://gertjanvandenburg.com/blog/autoencoder/
# https://github.com/GjjvdBurg/TensorFlowExperiments/tree/master/autoencoder

# https://github.com/GjjvdBurg/TensorFlowExperiments/blob/master/autoencoder/Makefile
!mkdir -p ./logs
!mkdir -p ./images
!ls -l

total 20996
drwxr-xr-x 2 root root     4096 Dec 14 07:45 images
drwxr-xr-x 2 root root     4096 Dec 14 07:45 logs
-rwxr-xr-x 1 root root 16117632 Jul 15  2017 ngrok
-rw-r--r-- 1 root root  5363700 Dec 14 07:45 ngrok-stable-linux-amd64.zip
drwxr-xr-x 1 root root     4096 Dec 10 17:34 sample_data


In [5]:
# Environment Setup for magenta
# https://colab.research.google.com/notebooks/magenta/performance_rnn/performance_rnn.ipynb#scrollTo=nzGyqJja7I0O
!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
!pip install -U magenta pyfluidsynth pretty_midi

# Hack to allow python to pick up the newly-installed fluidsynth lib.
import ctypes.util
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return ctypes.util.find_library(lib)

ctypes.util.find_library = proxy_find_library

# Download Salamander piano SoundFont.
# Samples by Alexander Holm: https://archive.org/details/SalamanderGrandPianoV3
# Converted to sf2 by John Nebauer: https://sites.google.com/site/soundfonts4u
!gsutil -m cp gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /tmp/

Selecting previously unselected package libfluidsynth1:amd64.
(Reading database ... 110377 files and directories currently installed.)
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
    100% |████████████████████████████████| 1.1MB 11.0MB/s 
Requirement already up-to-date: pretty_midi in /usr/local/lib/python3.6/dist-packages (0.2.8)
  Found existing installation: magenta 0.3.19
    Uninstalling magenta-0.3.19:
      Successfully uninstalled magenta-0.3.19
Copying gs://download.magenta.tensorflow.org/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2...
/ [1/1 files][591.9 MiB/591.9 MiB] 100% Done  46.7 MiB/s ETA 00:00:00           
Operation completed over 1 objects/591.9 MiB.                                    


In [6]:
#!/usr/bin/env python

"""TensorFlow MNIST AutoEncoder

This is my attempt to write the autoencoder for MNIST by Andrej Karpathy using 
ConvNetJS in TensorFlow. Mostly to get some more experience working in 
Tensorflow.

Sources:
    - http://cs.stanford.edu/people/karpathy/convnetjs/demo/autoencoder.html
    - https://www.tensorflow.org/get_started/mnist/pros

Author: Gertjan van den Burg
Date: Thu Oct 26 16:49:29 CEST 2017

"""

import numpy as np
import tensorflow as tf

from magenta.models.image_stylization.image_utils import form_image_grid
from tensorflow.examples.tutorials.mnist import input_data

BATCH_SIZE = 50
GRID_ROWS = 5
GRID_COLS = 10
USE_RELU = True


def weight_variable(shape):
    # From the mnist tutorial
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def fc_layer(previous, input_size, output_size):
    W = weight_variable([input_size, output_size])
    b = bias_variable([output_size])
    return tf.matmul(previous, W) + b


def autoencoder(x):
    # first fully connected layer with 50 neurons using tanh activation
    l1 = tf.nn.tanh(fc_layer(x, 28*28, 50))
    # second fully connected layer with 50 neurons using tanh activation
    l2 = tf.nn.tanh(fc_layer(l1, 50, 50))
    # third fully connected layer with 2 neurons
    l3 = fc_layer(l2, 50, 2)
    # fourth fully connected layer with 50 neurons and tanh activation
    l4 = tf.nn.tanh(fc_layer(l3, 2, 50))
    # fifth fully connected layer with 50 neurons and tanh activation
    l5 = tf.nn.tanh(fc_layer(l4, 50, 50))
    # readout layer
    if USE_RELU:
        out = tf.nn.relu(fc_layer(l5, 50, 28*28))
    else:
        out = fc_layer(l5, 50, 28*28)
    # let's use an l2 loss on the output image
    loss = tf.reduce_mean(tf.squared_difference(x, out))
    # http://testpy.hatenablog.com/entry/2016/11/27/035033
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(out, 1), tf.argmax(x, 1)), tf.float32))
    # https://qiita.com/mytk0u0/items/d11ce0c7393e96149fa6
    # accuracy, update_op = tf.metrics.accuracy(x, out)
    return loss, out, l3, accuracy


def layer_grid_summary(name, var, image_dims):
    prod = np.prod(image_dims)
    grid = form_image_grid(tf.reshape(var, [BATCH_SIZE, prod]), [GRID_ROWS, 
        GRID_COLS], image_dims, 1)
    return tf.summary.image(name, grid)


def create_summaries(loss, x, latent, output, accuracy):
    writer = tf.summary.FileWriter("./logs")
    tf.summary.scalar("Loss", loss)
    tf.summary.scalar("Accuracy", accuracy)
    layer_grid_summary("Input", x, [28, 28])
    layer_grid_summary("Encoder", latent, [2, 1])
    layer_grid_summary("Output", output, [28, 28])
    return writer, tf.summary.merge_all()


def make_image(name, var, image_dims):
    prod = np.prod(image_dims)
    grid = form_image_grid(tf.reshape(var, [BATCH_SIZE, prod]), [GRID_ROWS, 
        GRID_COLS], image_dims, 1)
    s_grid = tf.squeeze(grid, axis=0)

    # This reproduces the code in: tensorflow/core/kernels/summary_image_op.cc
    im_min = tf.reduce_min(s_grid)
    im_max = tf.reduce_max(s_grid)

    kZeroThreshold = tf.constant(1e-6)
    max_val = tf.maximum(tf.abs(im_min), tf.abs(im_max))

    offset = tf.cond(
            im_min < tf.constant(0.0),
            lambda: tf.constant(128.0),
            lambda: tf.constant(0.0)
            )
    scale = tf.cond(
            im_min < tf.constant(0.0),
            lambda: tf.cond(
                max_val < kZeroThreshold,
                lambda: tf.constant(0.0),
                lambda: tf.div(127.0, max_val)
                ),
            lambda: tf.cond(
                im_max < kZeroThreshold,
                lambda: tf.constant(0.0),
                lambda: tf.div(255.0, im_max)
                )
            )
    s_grid = tf.cast(tf.add(tf.multiply(s_grid, scale), offset), tf.uint8)
    enc = tf.image.encode_jpeg(s_grid)

    fwrite = tf.write_file(name, enc)
    return fwrite


def main():
    # initialize the data
    mnist = input_data.read_data_sets('/tmp/MNIST_data')

    # placeholders for the images
    x = tf.placeholder(tf.float32, shape=[None, 784])

    # build the model
    loss, output, latent, accuracy = autoencoder(x)

    # and we use the Adam Optimizer for training
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    # We want to use Tensorboard to visualize some stuff
    writer, summary_op = create_summaries(loss, x, latent, output, accuracy)

    first_batch = mnist.train.next_batch(BATCH_SIZE)

    # Run the training loop
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(make_image("images/input.jpg", x, [28, 28]), feed_dict={x : 
            first_batch[0]})
        for i in range(int(200001)):
            batch = mnist.train.next_batch(BATCH_SIZE)
            feed = {x : batch[0]}
            if i % 500 == 0:
                summary, train_loss, acc = sess.run([summary_op, loss, accuracy], 
                        feed_dict=feed)
                print("step %d, training loss: %g acc:%g" % (i, train_loss, acc))

                writer.add_summary(summary, i)
                writer.flush()

            if i % 1000 == 0:
                sess.run(make_image("images/output_%06i.jpg" % i, output, [28, 
                    28]), feed_dict={x : first_batch[0]})

            train_step.run(feed_dict=feed)

        # Save latent space
        pred = sess.run(latent, feed_dict={x : mnist.test._images})
        pred = np.asarray(pred)
        pred = np.reshape(pred, (mnist.test._num_examples, 2))
        labels = np.reshape(mnist.test._labels, (mnist.test._num_examples, 1))
        pred = np.hstack((pred, labels))
        if USE_RELU:
            fname = "latent_relu.csv"
        else:
            fname = "latent_default.csv"
        np.savetxt(fname, pred)


if __name__ == '__main__':
    main()

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __p

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
step 0, training lo